# 3. Diversity

(Remember to unpause the previous notebook)

So far, we have:
- separated and cleaned our samples, and 
- determined which representative sequences are in our sample and how many

This notebook tries to figure out how diverse our samples are

<img src="../assets/img/qiime_map.svg"  width="1200" height="600">

We start with three files from the previous notebook:
- `sample-metadata.tsv`: experimental design data from our samples
- `rep-seqs.qza`: the actual sequences of our OTU/ASVs
- `table.qza`: the frequencies of each OTU and sample.

## 4. Diversity analysis

We want to know:
- the diversity of each sample (alpha diversity)
- and how diverse are each pair of samples (beta diversity)

But before doing these analyses, we need to make all the sequences comparable.

### 4.1 Alignment and tree construction

To do so we are going to:
- align all the sequences
- compute their phylogenetic tree

Thankfully, we can do it in one go with `qiime phylogeny`

In [1]:
qiime phylogeny --help

Usage: qiime phylogeny [OPTIONS] COMMAND [ARGS]...

  Description: This QIIME 2 plugin supports generating and manipulating
  phylogenetic trees.

  Plugin website: https://github.com/qiime2/q2-phylogeny

  Getting user support: Please post to the QIIME 2 forum for help with this
  plugin: https://forum.qiime2.org

Options:
  --version            Show the version and exit.
  --example-data PATH  Write example data and exit.
  --citations          Show citations and exit.
  --help               Show this message and exit.

Commands:
  align-to-tree-mafft-fasttree  Build a phylogenetic tree using fasttree and
                                mafft alignment
  align-to-tree-mafft-iqtree    Build a phylogenetic tree using iqtree and
                                mafft alignment.
  align-to-tree-mafft-raxml     Build a phylogenetic tree using raxml and
                                mafft alignment.
  fasttree                      Construct a phylogenetic tree with FastTree.
  filter-tabl

There are three methods to align and build the tree:
- MAFFT + FastTree (fastest)  <- this
- MAFFT + IQTREE
- MAFFT + RAxML (most precise)

Let's see the help to know what we need to give it in order to work:

In [2]:
qiime phylogeny align-to-tree-mafft-fasttree --help

Usage: qiime phylogeny align-to-tree-mafft-fasttree [OPTIONS]

  This pipeline will start by creating a sequence alignment using MAFFT, after
  which any alignment columns that are phylogenetically uninformative or
  ambiguously aligned will be removed (masked). The resulting masked alignment
  will be used to infer a phylogenetic tree and then subsequently rooted at
  its midpoint. Output files from each step of the pipeline will be saved.
  This includes both the unmasked and masked MAFFT alignment from q2-alignment
  methods, and both the rooted and unrooted phylogenies from q2-phylogeny
  methods.

Inputs:
  --i-sequences ARTIFACT FeatureData[Sequence]
                          The sequences to be used for creating a fasttree
                          based rooted phylogenetic tree.           [required]
Parameters:
  --p-n-threads VALUE Int % Range(1, None) | Str % Choices('auto')
                          The number of threads. (Use `auto` to automatically
                        

In [3]:
qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences        rep-seqs.qza \
    --o-alignment        phylo-aligned-seqs.qza \
    --o-masked-alignment phylo-masked-aligned-seqs.qza \
    --o-tree             phylo-unrooted-tree.qza \
    --o-rooted-tree      phylo-rooted-tree.qza

Saved FeatureData[AlignedSequence] to: phylo-aligned-seqs.qza
Saved FeatureData[AlignedSequence] to: phylo-masked-aligned-seqs.qza
Saved Phylogeny[Unrooted] to: phylo-unrooted-tree.qza
Saved Phylogeny[Rooted] to: phylo-rooted-tree.qza



All the files are artifacts, so there is nothing to see :(

## 4.2 Core metrics

Also, the alpha and beta diversities are computed in one single command. It is done with `qiime diversity core-metrics-phylogenetic`:

Briefly:
- alpha: individual diversity of each sample
- beta: comparison of diversity between two samples

In [4]:
qiime diversity core-metrics-phylogenetic --help

Usage: qiime diversity core-metrics-phylogenetic [OPTIONS]

  Applies a collection of diversity metrics (both phylogenetic and non-
  phylogenetic) to a feature table.

Inputs:
  --i-table ARTIFACT FeatureTable[Frequency]
                          The feature table containing the samples over which
                          diversity metrics should be computed.     [required]
  --i-phylogeny ARTIFACT  Phylogenetic tree containing tip identifiers that
    Phylogeny[Rooted]     correspond to the feature identifiers in the table.
                          This tree can contain tip ids that are not present
                          in the table, but all feature ids in the table must
                          be present in this tree.                  [required]
Parameters:
  --p-sampling-depth INTEGER
    Range(1, None)        The total frequency that each sample should be
                          rarefied to prior to computing diversity metrics.
                                           

Remember [table.qzv](https://view.qiime2.org/visualization/?type=html&src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Ftable.qzv)

Here we set the `--p-sampling-depth` parameter to 1103. This value was chosen based on the number of sequences in the L3S313 sample because it’s close to the number of sequences in the next few samples that have higher sequence counts, and because it is considerably higher (relatively) than the number of sequences in the samples that have fewer sequences. 

In [5]:
export UNIFRAC_USE_GPU=N  # There is an error if you have a GPU
qiime diversity core-metrics-phylogenetic \
    --i-phylogeny      phylo-rooted-tree.qza \
    --i-table          table.qza \
    --p-sampling-depth 1103 \
    --m-metadata-file  sample-metadata.tsv \
    --output-dir       metrics

Saved FeatureTable[Frequency] to: metrics/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: metrics/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: metrics/evenness_vector.qza
Saved DistanceMatrix to: metrics/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: metrics/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: metrics/jaccard_distance_matrix.qza
Saved DistanceMatrix to: metrics/bray_curtis_distance_matrix.qza
Saved PCoAResults to: metrics/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: metrics/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: metrics/jaccard_pcoa_results.qza
Saved PCoAResults to: metrics/bray_curtis_pcoa_results.qza
Saved Visualization to: metrics/unweighted_unifrac_emperor.qzv
Saved Visualization to: metrics/weighted_unifrac_emperor.qzv
Saved Visualization to: metrics/jac

The results appear in the diversity-core-metrics-results folder

The visualizable ones:

- [metrics/bray_curtis_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray_curtis_emperor.qzv)
- [metrics/jaccard_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fjaccard_emperor.qzv)
- [metrics/unweighted_unifrac_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted_unifrac_emperor.qzv)
- [metrics/weighted_unifrac_emperor.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fweighted_unifrac_emperor.qzv)

For every distance, we have a clustering result:
- Bray-Curtis
- Jaccard
- Unweighted UNIFRAC
- Weighted UNIFRAC

## 4.3 Alpha diversity

Alpha diversities come in two flavors: 
- Richness (Faith Phylogenetic Diversity): how many ASVs are in the sample
- Evenness: how uniform is the distribution
We compute and visualize both:

In [6]:
# Richness ~ faith phylogenetic diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/faith_pd_vector.qza \
  --m-metadata-file   sample-metadata.tsv \
  --o-visualization   metrics/faith-pd-group-significance.qzv

Saved Visualization to: metrics/faith-pd-group-significance.qzv



In [7]:
# evenness
qiime diversity alpha-group-significance \
  --i-alpha-diversity metrics/evenness_vector.qza \
  --m-metadata-file   sample-metadata.tsv \
  --o-visualization   metrics/evenness-group-significance.qzv

Saved Visualization to: metrics/evenness-group-significance.qzv



- [metrics/faith-pd-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Ffaith-pd-group-significance.qzv)
- [metrics/evenness-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fevenness-group-significance.qzv)

### Exercise

**Note**: the Kruskal Wallis statitstical test checks if two or more samples come from the same distribution:
- H high -> p low -> they are different
- H low -> p high -> they are **not** different (!= equal)

- According to the Kruskal-Wallis test of all the samples, do they have the same or different **richness**?
  - By body site?
  - By subject?
  - By antibiotic usage?

- Body site: Different: p = 0.00044528728137249946 < 0.05
- Subject: Same: p = 0.2683816272927587 > 0.05
- Antibiotic usage: Different p = 0.018773411069677226 < 0.05

- According to the Kruskal-Wallis test of all the samples, do they have the same or different **evenness**?
  - By body site?
  - By subject?
  - By antibiotic usage?

- Body site: Different: p = 0.01916821252090555 < 0.05
- Subject: Same: p = 0.6926327840419735 > 0.05
- Antibiotic usage: Same: p = 0.14705851921929175 > 0.05

## 4.4 Beta diversity

We are going to analyze the sample composition using PERMANOVA tests. The purpose is to compare distances between groups of samples (body parts) and tell if they are different or not.

We should expect to see that the left and right hands are similar, that they are far away from gut, and that the tongue sits in beween

We are doing two tests: by body site and subject:

In [8]:
qiime diversity beta-group-significance \
    --i-distance-matrix metrics/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file   sample-metadata.tsv \
    --m-metadata-column body-site \
    --o-visualization   metrics/unweighted-unifrac-body-site-significance.qzv \
    --p-pairwise

Saved Visualization to: metrics/unweighted-unifrac-body-site-significance.qzv



In [9]:
qiime diversity beta-group-significance \
    --i-distance-matrix metrics/unweighted_unifrac_distance_matrix.qza \
    --m-metadata-file   sample-metadata.tsv \
    --m-metadata-column subject \
    --o-visualization   metrics/unweighted-unifrac-subject-group-significance.qzv \
    --p-pairwise

Saved Visualization to: metrics/unweighted-unifrac-subject-group-significance.qzv



- [metrics/unweighted-unifrac-body-site-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-body-site-significance.qzv)
- [metrics/unweighted-unifrac-subject-group-significance.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-subject-group-significance.qzv)

### Exercise

- Are there differences between body sites?

The all vs all analysis says that the p-value is 0.001 < 0.05, and therefore, the four samples don't come from a single distribution.

Pair-wise, The q-values are all below 0.05, with the exception of the left and right hands, which obviously should have the same microbiome.

- Are there differences between subjects?

Since it is a 1 vs 1 analysis, the group-wise and pair-wise analysis are the same. The p-value is 0.526 > 0.05, and therefore both subjects contain the same overall community composition.

Instead of using statistical tests and p-values to see the differences, we can use the `emperor` plugin to plot the samples in 3D space, and across the column `days-since-experiment-start`.

In [10]:
qiime emperor plot \
    --i-pcoa          metrics/unweighted_unifrac_pcoa_results.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-custom-axes   days-since-experiment-start \
    --o-visualization metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv

Saved Visualization to: metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv



In [11]:
qiime emperor plot \
    --i-pcoa          metrics/bray_curtis_pcoa_results.qza \
    --m-metadata-file sample-metadata.tsv \
    --p-custom-axes   days-since-experiment-start \
    --o-visualization metrics/bray-curtis-emperor-days-since-experiment-start.qzv

Saved Visualization to: metrics/bray-curtis-emperor-days-since-experiment-start.qzv



- [metrics/unweighted-unifrac-emperor-days-since-experiment-start.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Fbray-curtis-emperor-days-since-experiment-start.qzv)
- [metrics/bray-curtis-emperor-days-since-experiment-start.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Fcore-metrics-results%2Funweighted-unifrac-emperor-days-since-experiment-start.qzv)

### Exercise

- Do each body site follow their own progression?

Yes, they follow their own path across time.

- Are the right and left hands simmilar?

Obviously, they should be having simmilar communities because they are both skin that sit less than a meter appart from each other.

- Do you notice something weird?

There is a left palm samlpe clustered with the ones from tongue, and also a right palm sample clustered with the ones from gut.

Given that most of the subjects got tested twice per day and that at day 0 we observe 3 samples of gut and tongue, they may probably have mislabeled the samples in the `sample-data.tsv`, or maybe they were swapped at the laboratory.

Given what we have seen on alpha and beta diversities, there is no way that those samples belong to a hand.


## 4.5 Alpha rarefaction

Sometimes you need to know if you have sequenced enough for every sample.

Rarefaction consists on doing the same analysis multiple times with different coverages

In our case, we want to know if we have captured all the richness for every sample.

To do so, we run the alpha diversity function with 500 reads per sample, then 1000, them 1500, and so on, and plot the results:

We will know that we have sequenced enough if we are getting the same diversity, i.e., the plot has plateaued.

In [12]:
qiime diversity alpha-rarefaction \
    --i-table         table.qza \
    --i-phylogeny     phylo-rooted-tree.qza \
    --p-max-depth     4000 \
    --m-metadata-file sample-metadata.tsv \
    --o-visualization metrics/alpha-rarefaction.qzv

Saved Visualization to: metrics/alpha-rarefaction.qzv



- [alpha_rarefaction.qzv](https://view.qiime2.org/?src=https%3A%2F%2Fdocs.qiime2.org%2F2023.2%2Fdata%2Ftutorials%2Fmoving-pictures%2Falpha-rarefaction.qzv)

### Exercise

Do we reach a pleateau in all samples?

- According to the `faith_pd` metric (richness), most of the samples (`barcode-id`) did reach a plateau with the exception of the red and pink one.
- According to the `shannon` index, all samples stay stable across the entire plot.
- According to the `observed_features` metric,  there is still little to be gained if we sequence more. The red sample (sample L4S63), could still need some additional sequencing. 

What if we set the maximum depth to 12,000? 

In [14]:
qiime diversity alpha-rarefaction \
    --i-table         table.qza \
    --i-phylogeny     phylo-rooted-tree.qza \
    --p-max-depth     12000 \
    --m-metadata-file sample-metadata.tsv \
    --o-visualization metrics/alpha-rarefaction-12000.qzv

Plugin error from diversity:

  Provided max_depth of 12000 is greater than the maximum sample total frequency of the feature_table (9820).

Debug info has been saved to /tmp/qiime2-q2cli-err-yi9ifw3s.log



: 1

Qiime complains because no sample has that many reads.

- And 8,000?

In [15]:
qiime diversity alpha-rarefaction \
    --i-table         table.qza \
    --i-phylogeny     phylo-rooted-tree.qza \
    --p-max-depth     8000 \
    --m-metadata-file sample-metadata.tsv \
    --o-visualization metrics/alpha-rarefaction-8000.qzv

Saved Visualization to: metrics/alpha-rarefaction-8000.qzv



It takes way longer to execute, but now it shows that the red sample stabilizes. Also, there are no information for some samples when the plot reaches the point in which it has to plot a sampling above its depth.

## End of notebook

In [ ]:
pause

Click the stop button before continuing to the next notebook